In [1]:
import keras
import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Flatten, Dense, Input, Dropout
from keras.layers import Conv2D, ZeroPadding2D
from keras.layers import MaxPooling2D
from keras.metrics import top_k_categorical_accuracy
from keras.optimizers import SGD
from keras import backend as K

Using TensorFlow backend.


In [2]:
def VGG11(input_shape, num_classes):
    
    if K.image_data_format() == 'channels_first':
        input_shape = input_shape[::-1]
    else:
        input_shape = input_shape

    model = Sequential()

    # Block 1
    model.add(ZeroPadding2D((1, 1), input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu', name='block1_conv1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu', name='block1_conv2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))
    
    # Block 2
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), activation='relu', name='block2_conv1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(128, (3, 3), activation='relu', name='block2_conv2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))
    
    # Block 3
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu', name='block3_conv1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu', name='block3_conv2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu', name='block3_conv3'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(256, (3, 3), activation='relu', name='block3_conv4'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))
    
    # Block 4
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu', name='block4_conv1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu', name='block4_conv2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu', name='block4_conv3'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu', name='block4_conv4'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))
     
    # Block 5
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu', name='block5_conv1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu', name='block5_conv2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu', name='block5_conv3'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(512, (3, 3), activation='relu', name='block5_conv4'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

In [3]:
train_datagen = ImageDataGenerator(
        samplewise_center=True,
        samplewise_std_normalization=True,
        horizontal_flip=True,
        vertical_flip=True)

test_datagen = ImageDataGenerator()

In [10]:
train_generator = train_datagen.flow_from_directory(
    '../data/data_small_train/',     
    class_mode='categorical',           
    target_size=(224, 224),
    batch_size=50)

validation_generator = test_datagen.flow_from_directory(
    '../data/data_small_test/',  
    class_mode='categorical',
    target_size=(224, 224),
    batch_size=50)

Found 2509 images belonging to 25 classes.
Found 614 images belonging to 25 classes.


In [11]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)

In [12]:
model = VGG11((224, 224, 3), 25)

In [13]:
def top_5_categorical_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=5)

In [14]:
sgd = SGD(lr=0.01, decay=0.0005, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy', top_5_categorical_accuracy])

In [15]:
model.fit_generator(
    generator=train_generator,
    steps_per_epoch=50,
    validation_data=validation_generator,
    validation_steps=12,
    epochs=100,
    callbacks=[tbCallBack],
    verbose=2)

Epoch 1/100
200s - loss: 3.1975 - acc: 0.0856 - top_5_categorical_accuracy: 0.3120 - val_loss: 3.9784 - val_acc: 0.0917 - val_top_5_categorical_accuracy: 0.3217
Epoch 2/100
189s - loss: 3.1698 - acc: 0.0848 - top_5_categorical_accuracy: 0.3199 - val_loss: 4.1861 - val_acc: 0.0887 - val_top_5_categorical_accuracy: 0.3369
Epoch 3/100
185s - loss: 3.1677 - acc: 0.0826 - top_5_categorical_accuracy: 0.3275 - val_loss: 3.6234 - val_acc: 0.1046 - val_top_5_categorical_accuracy: 0.3280
Epoch 4/100
185s - loss: 3.1591 - acc: 0.0908 - top_5_categorical_accuracy: 0.3351 - val_loss: 4.4480 - val_acc: 0.0798 - val_top_5_categorical_accuracy: 0.3138
Epoch 5/100
185s - loss: 3.1644 - acc: 0.0894 - top_5_categorical_accuracy: 0.3220 - val_loss: 5.1103 - val_acc: 0.0851 - val_top_5_categorical_accuracy: 0.3085
Epoch 6/100
185s - loss: 3.1580 - acc: 0.0908 - top_5_categorical_accuracy: 0.3279 - val_loss: 6.7585 - val_acc: 0.0957 - val_top_5_categorical_accuracy: 0.3316
Epoch 7/100
185s - loss: 3.1645 - 

KeyboardInterrupt: 

In [11]:
model.save_weights('model_data_small_weights.h5')